# (To Be Deleted) Markdown Stylization (note spacing)
## Chapter = 2 (Number_Shift3) (note spacing)
### Subtitle = 3 (Number_Shift3) (note spacing)
*Italicized* = 1 (Star_Shift8) around

**Bold** = 2 (Star_Shift8) around OR as #### Heading (note spacing)

***Italicized & Bold*** = 3 (Star_Shift8) around

`Gray Square` = 1 (GraveAccent_NoShift~) around

Y (Code Type) ↔ M (Markdown Type)

> Gray Bar = 1 (GreaterThan_Shift.) before
>> Gray Bar = 2 (GreaterThan_Shift.) before, under above to be in same set
>>> Gray Bar = 3 (GreaterThan_Shift.)

    Indentation is always 4 (Spaces) before, under above to be in same set; changes the font
 
**▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬**

In [ ]:
#Importing our data and setting the "Id" column as our index
df = pd.read_csv('kc_house_data.csv', index_col = 'Id')

In [ ]:
built = (dup.groupby("Id")).Yr_Built.mean()
renovated = (dup.groupby("Id")).Yr_Renovated.mean()

average = filter(lambda x: x > 0, renovated)

oldest = df.loc[(df.Yr_Built == df.Yr_Built.min()) & (df.Id.duplicated(False))]

display(oldest)


print(round(built.mean(), ), built.min(), built.max())
print(renovated.count(), min(average), renovated.max())
print("")
print(oldest.Date.min().date())

#print("The oldest house resold was first built in", oldest.first().mean(), "and sold", oldest.last().mean(),"years later!")

In [ ]:
#Seeing our reduced DataFrame's descriptive stats and preview
display(df.info(), 
        pd.DataFrame(data = df.nunique().sort_values(), columns = ['Counts']).transpose(),
        df.describe(), 
        df)

In [ ]:
plt.title("Comparing predicted price vs actual price", fontdict={'fontsize': 20}), plt.xlabel("Values"), plt.ylabel("Prices")

In [ ]:
#Running a train-test-split model validation with our selected features
def linear_regression(X, y):
    
    train_err = []
    test_err = []
    sample = y.sample(n=200) #Generating 200 random rows in a sample list
    for i in sample: #i = individual house row
        temp_train_err = []
        temp_test_err = []
        #Performing the test 100 times to account for any particularly good/bad models that might have might have resulted from poor/good splits in the data
        for model in range(100):       
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123) #Seed for reproducibility
            linreg = LinearRegression()
            linreg.fit(X_train, y_train) #Fitting the model to the train data
            y_hat_train = linreg.predict(X_train) #Calculating predictions
            y_hat_test = linreg.predict(X_test) #y_pred
            temp_train_err.append(mean_squared_error(y_train, y_hat_train)) #Calculating the MSE
            temp_test_err .append(mean_squared_error(y_test, y_hat_test))
        train_err.append(np.mean(temp_train_err))
        test_err.append(np.mean(temp_test_err))
    
    #Outputting the average errors of 100 iterations of a random selected sample of 200 houses
    print("Average Errors on 100 Models of Random 200 Sampled Houses")
    print("R^2 score:", round(linreg.score(X, y), 2))
    print("Mean absolute error (MSE):", round(np.sum(test_err) / len(test_err), 5))
    print("Root MSE:", round(np.sqrt(np.sum(test_err) / len(test_err)), 5)) 
    print("-----------------------------------")
    print("Average predicted price:", round(y_hat_test.mean(), 4))
    print("Average actual price", round(y_test.mean(), 4))
    
    #Visualizing the averages of 100 iterations of a random selected sample of 200 houses
    plt.figure(figsize = (16, 6))
    sns.lineplot(sample, y_hat_test, label = "Predicted prices")
    sns.lineplot(sample, y_test, label = "Actual prices")
    plt.title("Comparing predicted price vs actual price"), plt.xlabel("House"), plt.ylabel("Prices")
    plt.legend(), plt.show()
    return (linreg, y_pred, y_test)

def cross_validation(X, y, cv=  15, show_scores = False):
    score = cross_val_score(linreg, X, y, cv = cv, scoring = "r2")
    print("Model Accuracy:", round(sum(score) / len(score), 2) * 100, "%")
    print(score) if show_scores else False
    
def feature_selection(X, y):
    est = sm.OLS(y, X).fit()
    pvalues = pd.DataFrame(est.pvalues, columns=["p"])
    features = list(pvalues[pvalues.p < 0.05].index)
    return features

In [ ]:
print(X.shape, y.shape, df.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print("")
print("")
print(X.info())
print("")
print("")
print(y.info())
print("")
print("")
print(df.info())
print("")
print("")
print(X_train.info())
print("")
print("")
print(X_test.info())
print("")
print("")
print(y_train.info())
print("")
print("")
print(y_test.info())

In [ ]:
#Separating our target and feature variables
X = df.drop(["Price"] , axis = 1)
y = pd.DataFrame(df.Price, columns=["Price"]) 

random.seed(123)
train_err = []
test_err = []
t_sizes = list(range(5, 100, 5)) #(Included start, excluded end, by size)
linreg = LinearRegression()
for t_size in t_sizes: #Generating 200 random samples of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = t_size / 100)
    linreg.fit(X_train, y_train ) #Fitting the model to the train data
    y_hat_train = linreg.predict(X_train) #Calculating predictions
    y_hat_test = linreg.predict(X_test)
    train_err.append(mean_squared_error(y_train, y_hat_train)) #Calculating the MSE
    test_err.append(mean_squared_error(y_test, y_hat_test))
print("Average Test Mean Squared Error:", np.sum(test_err)/len(test_err))
plt.figure(figsize = (16, 6))
plt.scatter(t_sizes, train_err, label="Training Error")
plt.scatter(t_sizes, test_err, label="Testing Error")
plt.title("Boston train-test-split on 1 Model")
plt.xlabel("Test Size (0 to 200)")
plt.ylabel("MSE")
plt.legend()
plt.show()

In [ ]:
#Counting function of unique values
display(df.Sqft_Basement.value_counts(), #Outputing as a raw number
        df.Sqft_Basement.value_counts(normalize=True)) #Outputing as a percentage